# Usage examples

The package offers routines for recreating temporal or spatiotemporal binary and Gaussian contrast values of white noise stimuli. The modules `binarystimulus` and `gaussianstimulus` are identical in their functions, function signatures, and return values. For simplicity, the code below shows examples for the binary stimulus module only. All examples work on the Gaussian stimuli analogously. Here, the module name is reduced to `stimulus` on import, for brevity.

In [1]:
from retinawhitenoise import binarystimulus as stimulus

All functions contain detailed doc-strings to help with their usage. Here an example

In [2]:
stimulus.recreate?

Signature: stimulus.recreate(seed, xy=(), num_frames=1, num_trials=1, progress=False)
Docstring:
Recreate temporal or spatiotemporal white noise stimulus for a given
set of parameters trial-by-trial to iterate over on the fly

Parameters
----------
seed : int or dict
    Initial seed

xy : tuple, optional
    Dimensions of the spatial frame, e.g. `(x, y)` for two spatial
    dimensions, `(s,)` for one spatial dimension, or `()` for no
    spatial component, that is, only temporal white noise, i.e. full
    field flicker. Default is `()`

num_frames : int, optional
    Number of spatial frames. Default is 1

num_trials : int, optional
    Number of trials. This determines the number of values to yield.
    Default is 1

progress : dict or bool, optional
    Arguments to pass to tqdm progress bar. If True, a tqdm progress
    bar is shown with its default properties. If False, no progress
    bar is shown. If dict, it serves as keyword arguments passed to
    tqdm. Default is False

Yiel

## Temporal ("full-field") stimulus

The following code recreates 5 trials of a temporal simulus, where one trial consists of 10 frames. Each iteration of the for-loop is one trial. Withing the for-loop, `stim` is a one-dimensional numpy array of size 10 containing the valuse of one trial at each iteration.

In [3]:
seed = -1000     # Seed as specified during the recording
xy = ()          # No spatial dimensions
num_frames = 10  # Ten values per trial
num_trials = 5   # Five trials = loop iterations

for stim in stimulus.recreate(seed, xy, num_frames, num_trials):
    print(stim)  # Each iteration prints one line

[-1 -1  1  1 -1  1  1  1  1  1]
[-1  1 -1  1 -1 -1 -1  1  1  1]
[-1 -1 -1 -1  1 -1  1 -1 -1 -1]
[-1  1  1  1  1  1  1 -1  1 -1]
[ 1  1  1 -1 -1  1  1 -1  1  1]


## Spatiotemporal stimulus

The following code recreates 5 trials of a spatiotemporal stimulus, each consisting of 10 frames of a spatial size of 80 by 50 (horizontal by vertical). Each iteration of the for-loop is one trial. Within the for-loop, `stim` is a $80 \times 50 \times 10$ numpy array representing one trial at each iteration.

In [4]:
xy = (80, 50)

for stim in stimulus.recreate(-1000, xy, 10, 5):
    assert stim.shape == (80, 50, 10)

## Save and load stimulus from disk

If a stimulus is used several times in an analysis, it may be beneficial to save and load the stimulus from disk instead. The function signature remains the same, with an additional parameter for the file path including a base file name. The stimulus of each trial is saved in one file. The file name of each trial is constructed from the base file name, e.g. `/path/to/basename`, followed by four digits (trial index) and a file extension, i.e. `[basename]0000.h5`.

In [5]:
basename = 'examples/running'  # Evaluates to examples/running0000.h5

In [6]:
stimulus.save(basename, seed, xy, num_frames, num_trials)

To load the stimulus from disk, pass the same base file name to the `load` function. This function works iteratively like the `recreate` function and can be used in the same way.

In [7]:
for stim in stimulus.load(basename):  # No trial number or file extension in the file name
    pass

## Load certain trials only

The critical benefit of loading the stimulus as opposed to recreating it on the fly is that certain sections can be loaded specifically. This may save time and memory. There are three ways to specify which trials to consider.

**Number of trials from start**

In [8]:
trials = 4  # Load trials 0 through 4

Using the line above will result in loading
>examples/running0000.h5  
>examples/running0001.h5  
>examples/running0002.h5  
>examples/running0003.h5

**Range of trials**

In [9]:
trials = range(2, 5)  # Load trials 2 through 5

Using the line above will result in loading
>examples/running0002.h5  
>examples/running0003.h5  
>examples/running0004.h5

**Indices of trials**

In [10]:
trials = [2, 5, 8]  # Load exactly trial 2, 5, and 8

Using the line above will result in loading
>examples/running0002.h5  
>examples/running0005.h5  
>examples/running0008.h5

This argument is passed in the following way:

In [11]:
for stim in stimulus.load(basename, num=trials):
    pass  # Only loading the specified trials

## Load certain spatial region only

Likewise, it may save time and memory to only load a region in the spatial stimulus that is relevant, e.g. containing a receptive field in question. This is achieved by specifying the edges in the spatial dimensions in a tuple. For two spatial dimensions, slices for x and y are specified like so:

In [12]:
window = (slice(23, 45), slice(12, 48))  # Rectangle of x (23 to 45) and y (12 to 48)

for stim in stimulus.load(basename, crop=window):
    assert stim.shape[:2] == (22, 36)  # Only loading a 22 x 36 spatial window

## Recreate or load one trial only

The functions `recreate` and `load` each return a generator, a concept in Python that can be explained as a "generating iterator". The values provided at each iteration (e.g. of the for-loops in the examples above) are generated on the fly, whether that is recreating the stimulus or loading it from disk. To `recreate` or `load` only *one* trial or file, both functions will still return a generator although there is only one iteration. To access the stimulus without writing an unnecessary for-loop, one can iterate the generation by wrapping the call in `next`.

In [13]:
stim = next(stimulus.load(basename, num=1))  # Only load one trial

## Progress bar

With many trials or large-scale processing, it may be useful to indicate the progress over trials. This can be achieved by passing `True` for the parameter `progress`. This turns on a tqdm progress bar with its default properties. This parameter is available in all three functions (`recreate`, `save`, and `load`).

In [14]:
for stim in stimulus.recreate(-1000, (200, 150), 20, 100, progress=True):
    pass  # Larger stimulus dimensions to illustrate the progress bar

When specifying a dictionary as argument instead, keyword arguments can be passed to tqdm to adjust the progress bar.

In [15]:
# Example properties
tqdm_args = dict(
    desc='Processing stimulus trials',  # Change the title of the progress bar
    leave=True,                         # Keep the progress bar after finishing
)

for stim in stimulus.recreate(-1000, (200, 150), 20, 100, tqdm_args):
    pass

Processing stimulus trials: 100%|█████████████████████████████████████████████████| 100/100 [00:03<00:00, 26.67trial/s]


Saving a stimulus to disk enables the progress bar by default. To suppress the progress bar, specify `False`.

In [16]:
stimulus.save(basename, seed, xy, num_frames, num_trials, False)

## Several stimulus parts

Sometimes a stimulus presentation is broken into several parts in an experiment (e.g. in chunks of one hour), sometimes initialized with a different seed (e.g. first hour `-1000`, second hour `-2000`).

In [17]:
# Recreate the stimulus of the second hour with a different seed
stimulus.save('examples/runningTwo', -2000, xy, num_frames, 3)

The created files in the `examples`-directory now contain five trials of `running` and three trials of `runningTwo`.
>examples/running0000.h5  
>examples/running0001.h5  
>examples/running0002.h5  
>examples/running0003.h5  
>examples/running0004.h5  
>examples/runningTwo0000.h5  
>examples/runningTwo0001.h5  
>examples/runningTwo0002.h5

On loading, these two (or more) chucks can be stitched-together to analyze them as one long stimulus sequence. They can be iterated over successively by [chaining generators](https://docs.python.org/3/library/itertools.html#itertools.chain).

In [18]:
from itertools import chain  # Allow chaining iterators

hour1 = 'examples/running'
hour2 = 'examples/runningTwo'

# Some progress bar information for illustration
p1 = dict(desc='Hour 1', leave=True)
p2 = dict(desc='Hour 2', leave=True)

# Iterate over all trials of hour1 and then of all trials of hour2
for stim in chain(stimulus.load(hour1, progress=p1), stimulus.load(hour2, progress=p2)):
    from time import sleep; sleep(0.25)  # Slow down progress for illustration

Hour 2: 100%|█████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.87trial/s]


## Advanced properties

#### File size
Saving the stimulus is done in HDF4 format (h5 file extension). The compression is set to 3 (out of 9) by default to allow fast loading while preserving some disk space. The value can be adjusted if necessary. See below.

#### Floating point precision
To save space in memory, the floating point values in the Gaussian white noise module are generated with 32-bit precesion. In case a precision of 64-bit is necessary, it can be adjusted. See below.

In [19]:
import retinawhitenoise
retinawhitenoise._core.COMPRESSION = 3  # H5 compression for writing stimulus to disk
retinawhitenoise._core.PRECISION = 'float32'  # Or 'float64' for Gaussian white noise

## Pseudo-random number generator

The values generated are based on `ran1` and `gasdev` from Numerical Recipes. If needed, the (pseudo-) random number generator (RNG) can be accessed directly.

In [20]:
from retinawhitenoise import Rng

The RNG is implemented in the class `Rng` that is initialized with a seed. The object-oriented design allows multiple instances with independent seeds.

In [21]:
r = Rng(-1000)

The methods `ran1` and `gasdev` then generate respective values and advance the seed. The methods return the values in Python lists.

In [22]:
r.ran1(5)

[0.19059444460579866,
 0.4645137076566525,
 0.928777586169903,
 0.5020774898594607,
 0.436494727822251]

In [23]:
r.gasdev(5)

[1.0777831931095547,
 1.0312520475132794,
 -0.6891975327514535,
 1.6333838175558486,
 -0.7822342155031711]

Here, `ranb` is added to provide a binary version of `ran1`. It is used in the implementation of `binarystimulus`.

In [24]:
r.ranb(5)

[1, 0, 0, 0, 1]